# Monster Hunter Freedom Unite
Uma interface onde o usuário posssa selecionar as informações de Monster Hunter Freedom Unite.
As seguintes informações devem ser acessadas pelo usuário:

    - Lista de Quests e as Key Quests.
        - Elder Quests
        - Nekoht Quests
        - Guild Hall Quests

    - Lista de Monstro:
        - Nome do Monstro em Inglês & Japones.
        - Classe do Monstro
        - Elemento & Fraqueza Elemental
        - Habitat
        - Geração
        - Frase*

    - Lista de Items:
        - Nome, Informação e Preço


In [2]:
import requests
import re

import numpy as np
import pandas as pd

from bs4 import BeautifulSoup
from pandas import DataFrame

# 1.0. Key Quest List

In [774]:
url = 'https://monsterhunter.fandom.com/wiki/MHFU:_Guild_Quests'
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5),AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}

page = requests.get( url, headers=headers )
soup = BeautifulSoup( page.text, 'html.parser' )

In [792]:
quest_list = [list( filter ( None, p.get_text().split('\n') ) ) for p in soup.find_all( 'table', class_='linetable hover2' )]
quest_list = pd.DataFrame( quest_list ).T

quest_board = [list( filter( None, p.get_text().split('\n') ) ) for p in soup.find_all( 'table', class_='linetable hover2' ) ]

quest_list = pd.DataFrame()

for i in range( len( quest_board ) ):
    quest_index = pd.DataFrame( [list( filter( None, p.get_text().split('\n') ) ) for p in soup.find_all( 'table', class_='linetable hover2' )[i].find_all('tr')] )
    quest_list  = pd.concat( [quest_list, quest_index], axis=0 )

In [793]:
quest_list.columns = ['title', 'goal', 'area', 'time', 'fee', 'reward', 'notes', 'nan', 'nan2']

quest_list = quest_list[~quest_list['title'].str.contains('Quest Title')]

quest_list = quest_list.reset_index()
quest_list = quest_list.drop( columns=['index', 'nan2'] )

In [794]:
df_ref = pd.DataFrame( index=np.arange( len( quest_list ) ) )

u_quests = quest_list[quest_list['title'] == 'URGENT:']
u_quests = u_quests.drop( columns=['title'] )
u_quests.columns = ['title', 'goal', 'area', 'time', 'fee', 'reward', 'notes']

# Title
df_ref = pd.concat( [df_ref, quest_list[~quest_list['title'].str.contains('URGENT')]['title']], axis=1 )
df_ref['title'] = df_ref['title'].combine_first( u_quests['title'] ) 


# goal
quest_goal = quest_list[~quest_list['goal'].str.contains('The Ruler of the Snow')]['goal']
list_remove = quest_list[quest_list['title'] == 'URGENT:']['goal'].tolist()

for i in list_remove:
    quest_goal = quest_goal[~quest_goal.str.contains(i)]
    
df_ref = pd.concat( [df_ref, quest_goal], axis=1)
 
df_ref['goal'] = df_ref['goal'].combine_first( u_quests['goal'] )


# area
quest_area = quest_list[~quest_list['area'].str.contains('Hunt the Blangonga')]['area']
list_remove = quest_list[quest_list['title'] == 'URGENT:']['area'].tolist()

for i in list_remove:
    quest_area = quest_area[~quest_area.str.contains(i)]
    
df_ref = pd.concat( [df_ref, quest_area], axis=1)
 
df_ref['area'] = df_ref['area'].combine_first( u_quests['area'] )


# time
quest_time = quest_list['time'].str.extract('(\d+\w+)')[0]
quest_time.name = 'time'
quest_time = quest_time.combine_first( u_quests['time'] )

df_ref = pd.concat( [df_ref, quest_time], axis=1 )
df_ref.loc[df_ref['time'] == '50', 'time'] = '50mins'


# fee
quest_fee = quest_list[~quest_list['fee'].str.contains('mins')]['fee']
quest_fee = quest_fee.combine_first( u_quests['fee'] )
quest_fee.name = 'fee'

df_ref = pd.concat( [df_ref, quest_fee], axis=1 )


# reward
quest_rewards = quest_list['reward']
quest_new_reward = quest_list[quest_list['title'] == 'URGENT:']['notes'].tolist()
index = [41, 68, 88, 102, 130, 165, 166, 167]

for i in range( len( index ) ):
    quest_rewards[quest_rewards.index[index[i]]] = quest_new_reward[i]
    
quest_rewards.name = 'reward'
df_ref = pd.concat( [df_ref, quest_rewards], axis=1 )


# notes
quest_notes = quest_list[~quest_list['title'].str.contains('URGENT:')]['notes']
quest_notes = quest_notes.combine_first( u_quests['notes'] )
quest_notes.name = 'notes'

df_ref = pd.concat( [df_ref, quest_notes], axis=1 )

df_ref['notes'] = df_ref['notes'].fillna('No Info')

df_ref.to_csv('data/all_quests.csv')

In [795]:
df_ref

,title,goal,area,time,fee,reward,notes
0,A True Foe - The Giadrome!,Hunt the Giadrome,Snowy Mountains (Day),50mins,100z,1200z,No Info
1,Reckless Bulldrome Hunter,Hunt the Bulldrome,Snowy Mountains (Night),50mins,100z,1500z,No Info
2,A Pack of Blangos,Slay 15 Blangos,Snowy Mountains (Day),50mins,100z,900z,Bulldrome is present
3,Mountain Herb Picking,Deliver 20 Mountain Herbs,Snowy Mountains (Night),50mins,100z,900z,Bulldrome is present
4,Rarest of Rare Beasts,Hunt the Congalala,Jungle (Night),50mins,150z,1800z,No Info
...,...,...,...,...,...,...,...
250,The Approaching Gaoren,Defend the Town from Shen Gaoren,Town,30mins,3500z,42000z,You will fail if the Fort Integrity reaches Ze...
251,Absolute Zero,Slay the Ukanlos,SnwyMtnsPeak,50mins,5000z,60000z,Lv G★★★ Urgent Quest
252,Legendary Black Dragon,Slay the Fatalis,Castle Schrade,50mins,4500z,54000z,Complete at least one weapon training (G-Class...
253,The End Times,Slay the Crimson Fatalis,Battleground,50mins,4500z,54000z,Slay the Fatalis and clear all Elder Dragon qu...


In [883]:
required_quests = df_ref[df_ref['notes'].str.contains('Urgent')]
required_quests = pd.concat( [required_quests, df_ref[df_ref['notes'].str.contains('Unlocks')]], axis=0 )

urgent_quests = required_quests[required_quests['notes'].str.contains('LV')]
urgent_quests = pd.concat( [urgent_quests, required_quests[required_quests['notes'].str.contains('Lv')]], axis=0 )
urgent_quests = urgent_quests[4:]

key_quests['rank'] = 1
key_quests.iloc[4:9, 7]   = 2
key_quests.iloc[9:13, 7]  = 3
key_quests.iloc[13:17, 7] = 4
key_quests.iloc[17:22, 7] = 5
key_quests.iloc[22:27, 7] = 6
key_quests.iloc[27:32, 7] = 7
key_quests.iloc[32:37, 7] = 8
key_quests.iloc[37:, 7]   = 9

key_quests = pd.concat( [key_quests, urgent_quests], axis=0 )
key_quests.iloc[-1:, 7]   = 9
key_quests.iloc[-2:-1, 7] = 8
key_quests.iloc[-3:-2, 7] = 7
key_quests.iloc[-4:-3, 7] = 5
key_quests.iloc[-5:-4, 7] = 4

key_quests = pd.concat( [key_quests, required_quests[required_quests['notes'].str.contains('Unlocks G')]], axis=0 )
key_quests.iloc[-1:, 7] = 6
key_quests

key_quests.to_csv('data/key_quests.csv')

In [884]:
key_quests

,title,goal,area,time,fee,reward,notes,rank
22,Giadrome Assault (Key Quest),Hunt the Giadrome,Snowy Mountains (Night),50mins,200z,2400z,Required for next Urgent,1.0
33,The Land Shark (Key Quest),Hunt the Cephadrome,Desert (Night),50mins,220z,2700z,Required for next Urgent,1.0
34,The Lurking Desert Giant (Key Quest),Hunt the Daimyo Hermitaur,Desert (Day),50mins,350z,4200z,Required for next Urgent,1.0
40,The Mischief Maker (Key Quest),Hunt the Congalala,Swamp (Night),50mins,350z,4200z,Required for next Urgent,1.0
50,Master of the Giant Lake,Hunt the Plesioth,Jungle (Night),50mins,450z,5400z,Required for next Urgent,2.0
...,...,...,...,...,...,...,...,...
130,Land of the Tremors,Hunt 2 Tigrex,Snowy Mountains (Night),50mins,1250z,15000z,Unlocks LV★★★★★★★★ quests; HR6 Rank-Up,5.0
193,Ever-Present Shadow,Hunt the Nargacuga,GrtForest (Night),50mins,1050z,12600z,Lv G★ Urgent Quest (For HR8),7.0
221,The Floating Dragon,Slay the Yama Tsukami,Tower,50mins,2500z,30000z,Lv G★★ Urgent Quest (For HR9),8.0
251,Absolute Zero,Slay the Ukanlos,SnwyMtnsPeak,50mins,5000z,60000z,Lv G★★★ Urgent Quest,9.0


# 2.0. Monster List

In [71]:
url = 'https://monsterhunter.fandom.com/wiki/MHFU:_Monsters'
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5),AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}

page = requests.get( url, headers=headers )
soup = BeautifulSoup( page.text, 'html.parser' )

# 3.0. List Itens

In [2]:
url = 'https://monsterhunter.fandom.com/wiki/MHFU:_Item_List'
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5),AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}

page = requests.get( url, headers=headers )
soup = BeautifulSoup( page.text, 'html.parser' )

In [3]:
all_itens = [list( filter( None, p.get_text().split('\n') ) ) for p in soup.find( 'div', class_='mw-parser-output' ).find_all('tr')]

In [125]:
item_data = DataFrame( all_itens )
item_data.columns = ['name', 'description', 'found', 'value', 'rarity']
item_data = item_data[1:]
item_data = item_data.drop_duplicates()
item_data = item_data[~item_data['name'].str.contains('Name')]
item_data = item_data.reset_index( drop=True )

# Create Dataframe for inserv Values
df_ref = DataFrame( index=np.arange( len( item_data ) ), columns=['name_', 'description_', 'found_', 'value_', 'rarity_'] )

# Name
df_ref = pd.concat( [df_ref, item_data['name']], axis=1 )

# Description
df_ref = pd.concat( [df_ref, item_data['description']], axis=1 )

# Found
item_data.loc[item_data['name'] == 'Armor Stone', 'found'] = 'Pokke Farm mining spots and bomb mining or Buy at the item shop for fifteen hundred'
item_found = item_data[~item_data['found'].str.contains('z')]['found']
df_ref = pd.concat( [df_ref, item_found], axis=1 )

# Item Values
item_value2 = item_data['found'].str.extract('(\d+z)')[0]
item_value2.name = 'value'
item_value = item_data[item_data['value'].str.contains('\d+.z')]['value']

item_value = item_value.combine_first( item_value2 )
df_ref = pd.concat( [df_ref, item_value], axis=1 )

# Item Rarity
item_rarity  = item_data[~item_data['value'].str.contains('z')]['value']
item_rarity2 = item_data['rarity']
item_rarity  = item_rarity2.combine_first( item_rarity )
df_ref = pd.concat( [df_ref, item_rarity], axis=1 )

df_ref = df_ref.drop( columns=['name_', 'description_', 'found_', 'value_', 'rarity_'], axis=1 )

# Fill Na Values
# Rarity
df_ref['rarity'] = df_ref['rarity'].fillna('2')

# Values
value_missing = item_data['value'].str.extract( '(^\dz$)' )[0]
df_ref['value'] = df_ref['value'].combine_first( value_missing )
df_ref['value'] = df_ref['value'].fillna( '23z' )

# Found
df_ref.loc[df_ref['name'].str.contains('Tckt'), 'found'] = df_ref.loc[df_ref['name'].str.contains('Tckt'), 'found'].fillna('Gathering Hall')
df_ref.loc[df_ref['name'].str.contains('Pill'), 'found'] = df_ref.loc[df_ref['name'].str.contains('Pill'), 'found'].fillna('Check combine list')
df_ref.loc[df_ref['name'].str.contains('Ancient Potion'), 'found'] = df_ref.loc[df_ref['name'].str.contains('Ancient Potion'), 'found'].fillna('Check combine list')

df_ref['found'] = df_ref['found'].fillna( 'Other Location...' )

# Change Dtypes
df_ref['rarity'] = df_ref['rarity'].astype('int64')

item_data.to_csv('/data/item.csv')

# 4.0. Mix Sets

In [129]:
mix_sets = DataFrame( columns = ['weapon', 'head', 'chest', 'hands', 'tasset', 'boots', 'skills'] )

In [ ]:
[['AncientDrgnwoodCan', 'Barrage Piercing', 'Kiaser Vest X', 'Conga Guards Z', 'Kaiser Coat X', 'Genesis Pants Goku', 'Reckless Abandon +3, Capacity Up, Pellet Up'],